<a href="https://colab.research.google.com/github/timbosssds/ML/blob/main/vectordb_retrive_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Want to set up a vector db, so I can try 'MultiQueryRetriever'

In [ ]:
# source: https://python.langchain.com/docs/use_cases/question_answering/

In [1]:
# # Install required libraries
#!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers
#!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers

# -- Only below run
!pip install langchain
!pip install pypdf
!pip install PyPDF2
!pip install chromadb
!pip install --upgrade chromadb
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.7/507.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# -- Not run
#import bs4
#from langchain import hub
#from langchain.chat_models import ChatOpenAI
#from langchain.document_loaders import WebBaseLoader
#from langchain.embeddings import OpenAIEmbeddings
#from langchain.schema import StrOutputParser
#from langchain.schema.runnable import RunnablePassthrough
#from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.vectorstores import Chroma

In [ ]:
# (Baseline - do not edit)
# MileStone 1: Ingest, chunk, embed, store in df.
# -- end-to-end (With Citations)
# -- 1.0 Load the PDF document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
#PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')
PDF_PATH = '/content/cba_fsg.pdf'
loader = PyPDF2.PdfReader(PDF_PATH)

# -- 2.0 Create an empty string to store the text
text = ""
# Loop through each page of the PDF
for page_num in range(len(loader.pages)):
    # Get the page object
    page = loader.pages[page_num]
    # Extract the texst from the page and add it to the text variable
    text += page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# Not using atm
# # -- 3.0 Process text
# def convert_list_to_string_with_speech_marks(list_items):
#   """Converts a list to a string with speech marks at each end."""
#   string = " ".join(list_items)
#   string = f"\"{string}\""
#   return string
# list_items = texts
# text = convert_list_to_string_with_speech_marks(list_items)


# ------------ new (embed & store)
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# Define the chunked text
chunked_text = texts

# Load the Sentence Transformer model
model = SentenceTransformer("paraphrase-mpnet-base-v2")

# Embed the chunked text
embeddings = model.encode(chunked_text)

# Convert embedding vectors into one-dimensional arrays
one_dimensional_embeddings = []
for embedding in embeddings:
    one_dimensional_embedding = np.ravel(embedding)
    one_dimensional_embeddings.append(one_dimensional_embedding)

# Combine the original text and embeddings into a DataFrame
data = {
    "original_text": chunked_text,
    "embeddings": one_dimensional_embeddings
}

df = pd.DataFrame(data)
display(df)

,original_text,embeddings
0,Financial \nServices \nGuide.\nDated 1 Octobe...,"[-0.0912222, -0.003018869, 0.038152117, -0.033..."
1,Services \nGuide.Our products and services\nO...,"[-0.033502605, -0.13524558, 0.0170168, -0.0555..."
2,When providing financial services that relate ...,"[-0.057890907, -0.12968607, 0.057180542, 0.018..."
3,Investments HoldCo Pty Limited ABN 64 644 660 ...,"[-0.0370728, -0.03843237, 0.02864407, 0.032806..."
4,Our staff are remunerated by salary that inclu...,"[-0.10325377, -0.19674908, -0.01208837, -0.006..."
5,"and cultural events, corporate promotional mer...","[0.0023660515, 0.072373606, 0.003049189, 0.025..."
6,"Insurance Partners Limited from us, the commis...","[-0.05016672, -0.054218188, 0.0806684, -0.0192..."
7,"Australian Prudential Regulation Authority, we...","[-0.041048624, 0.14313406, -0.072433785, 0.100..."
8,important we hear about it so we can make thin...,"[-0.08806811, -0.04257054, -0.017361928, -0.03..."
9,You will be advised of the details (including ...,"[-0.15787879, 0.014245284, 0.00045931916, 0.07..."


In [ ]:
model = "deepset/roberta-base-squad2"

In [2]:
# (WIP of above Baseline - edit)
# MileStone 1: Ingest, chunk, embed, store in df.
# -- end-to-end (With Citations)
# -- 1.0 Load the PDF document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
import pandas as pd
pd.set_option('display.max_colwidth', None)
PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')
#PDF_PATH = '/content/cba.pdf'
loader = PyPDF2.PdfReader(PDF_PATH)

# -- 2.0 Create an empty string to store the text
text = ""
# Loop through each page of the PDF
for page_num in range(len(loader.pages)):
    # Get the page object
    page = loader.pages[page_num]
    # Extract the texst from the page and add it to the text variable
    text += page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# Not using atm
# # -- 3.0 Process text
# def convert_list_to_string_with_speech_marks(list_items):
#   """Converts a list to a string with speech marks at each end."""
#   string = " ".join(list_items)
#   string = f"\"{string}\""
#   return string
# list_items = texts
# text = convert_list_to_string_with_speech_marks(list_items)


# ------------ new (embed & store)
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# Define the chunked text
chunked_text = texts

# Load the Sentence Transformer model
model = SentenceTransformer("paraphrase-mpnet-base-v2")

# Embed the chunked text
embeddings = model.encode(chunked_text)

# Convert embedding vectors into one-dimensional arrays
one_dimensional_embeddings = []
for embedding in embeddings:
    one_dimensional_embedding = np.ravel(embedding)
    one_dimensional_embeddings.append(one_dimensional_embedding)

# Combine the original text and embeddings into a DataFrame
data = {
    "original_text": chunked_text,
    "embeddings": one_dimensional_embeddings
}

df_corpus = pd.DataFrame(data)
display(df_corpus.head(2))

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

,original_text,embeddings
0,"Financial \nServices \nGuide.\nDated 1 October 2022\nWe are required under the Corporations Act to \nprovide you with the following information:\nThis Financial Services Guide (FSG) is issued by \nCommonwealth Bank of Australia (‘Bank’, ‘we’, \n‘our’ or ‘us’) and is intended to provide you with \ninformation that will assist you in making informed \ndecisions on whether you want to use our products \nand services. It includes details about t he following:\n• financial services we are able to pr ovide to you;\n• how we and our staff are paid for p roviding you \nwith th ose services;\n• how to co ntact us; and\n• the internal and external dispute resolution \nsystems you can use if you have a complaint to \nmake about us.\nNot Independent\nOur employees who give personal advice \nto retail clients about deposit products and \nrelated payment facilities are not independent, \nimpartial or unbiased because our employees \nare employed and remunerated by us and can \nonly advise on our products.\nFinancial","[-0.0912222, -0.003018869, 0.038152117, -0.033144977, 0.15137935, 0.052747663, 0.116495445, -0.017546244, 0.16356567, -0.039303206, 0.008831106, -0.222809, 0.1475248, 0.10988063, -0.09048118, -0.07642102, -0.023956433, -0.04330139, 0.045893997, 0.01381569, 0.09996568, -0.08102244, 0.054413985, -0.034748085, -0.1959137, -0.21640666, 0.011971034, 0.03803204, -0.015266558, 0.2041085, 0.17807388, 0.049132984, -0.00058644894, 0.041973323, 0.020010252, -0.11688024, -0.015840996, -0.0029988023, -0.16008875, 0.15786241, 0.051001206, -0.034563184, 0.012300072, 0.0014560283, -0.13889997, 0.010499321, -0.10086365, 0.07077726, 0.17997842, 0.16796248, 0.038596015, -0.20325251, -0.046487775, -0.0967121, -0.123048015, -0.030472748, -0.003693951, 0.020497255, -0.23838793, 0.058730565, -0.04794224, -0.020663586, -0.103670806, 0.016580792, -0.060577467, -0.037270743, -0.01448723, 0.08673652, -0.13492177, -0.04582237, 0.4423343, 0.016728092, -0.14585765, 0.06276913, 0.090604715, 0.2608602, 0.045893304, 0.12504196, -0.032472167, -0.038698956, -0.022524308, -0.022383515, -0.116810076, -0.085512385, -0.09689088, -0.060900863, -0.04210629, 0.05709929, 0.035738923, 0.054541074, -0.28274107, 0.06850003, -0.100981064, -0.08784868, 0.052980002, 0.1271797, -0.055376697, 0.015372893, 0.19833753, -0.067349054, ...]"
1,"Services \nGuide.Our products and services\nOur Australian Financial Services Licence authorises us to deal \nin and provide financial product advice relating to the following \nproduct groups:\n• Banking products, such as deposit accounts and payme nt \nfacilities\n• Gener al Insurance \n• Life Insurance and Income Protection (including Risk \nInsurance and Investment Li fe insurance)\n• Foreign Exchange (including Trave l Money Card)\n• Superannuation\nOur representatives may provide:\n• personal advice on banking products, such as deposit \naccounts and payment facilities. \n• general advice on General Insurance, Life Insurance, Foreign \nExchange and Superannuation. If we provide you with general \nadvice on these products we will also give you a Product \nDisclosure Statement (PDS), which gives you information \nabout the product on which we pr ovide advice \nand helps you to compare with simi lar products.\nAssociations or relationships with other product issuers","[-0.033502605, -0.13524558, 0.0170168, -0.055539966, 0.16302902, 0.034889635, 0.31856266, -0.033092856, 0.10857379, -0.07262491, -0.03907413, -0.18781881, 0.10539302, 0.1812964, 0.086593606, -0.2454868, 0.06041441, -0.079782926, 0.24021415, 0.06991153, 0.069081455, -0.07149693, 0.035200927, 0.053934693, -0.040603068, -0.15668663, -0.018495737, 0.00397027, 0.09342651, 0.18159305, 0.056351487, -0.054133974, 0.11070648, -0.13251022, 0.042476047, -0.033585127, -0.040889654, -0.03878325, -0.056411576, 0.11202981, 0.121330544, 0.08659795, 0.09803924, 0.024736507, -0.079058565, 0.024097245, 0.0131386155, 0.08377803, 0.17437004, 0.13847049, -0.025934773,

In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

# Initialize the Sentence Transformer model
model = SentenceTransformer("paraphrase-mpnet-base-v2")

# Define the question
question = "what is the date of this document"

# Embed the question
embeddings = model.encode([question])

# Flatten the embedding list into a one-dimensional array
one_dimensional_embeddings = np.concatenate(embeddings).ravel()

# Convert array to a single-row dataframe
df_question = pd.DataFrame({'embeddings': [one_dimensional_embeddings]})

df_question['question'] = question
df_question = df_question[['question','embeddings']]
display(df_question)

,question,embeddings
0,what is the date of this document,"[0.040262967, 0.08659528, 0.12466674, -0.01663203, -0.12787767, -0.025562787, 0.031162905, -0.051343713, -0.03529166, 0.12506978, 0.22244185, -0.06107027, 0.0375653, -0.20011333, -0.0052904296, -0.08712422, -0.055732157, 0.11806301, -0.037637133, 0.015782421, -0.009139829, -0.041441057, 0.029501626, 0.025478462, -0.14328434, -0.089592226, -0.1796084, 0.08220087, -0.14660138, 0.004710068, -0.03031806, 0.11280808, 0.14858697, 0.1661192, 0.052468408, -0.13023975, -0.09549001, 0.0067625763, -0.094102286, 0.17805414, 0.20880872, -0.0977463, -0.19663744, -0.16396476, -0.3184132, -0.12877081, -0.09599057, 0.13399917, -0.04202567, 0.3196744, 0.12947083, 0.12844166, 0.3446759, -0.042895705, 0.1127204, -0.08722481, -0.11206044, 0.0033633956, 0.06549252, 0.027309645, 0.042235766, 0.09082972, -0.01516528, -0.1657499, -0.14257781, -0.24741763, 0.14735635, -0.016002273, -0.11382242, -0.08948267, 0.50571406, 0.016203735, -0.10415986, 0.30918294, -0.018244095, -0.08824416, -0.0017436577, -0.01892244, 0.15014361, -0.095726356, 0.2721145, 0.015913555, -0.14161408, 0.06843128, -0.010548831, 0.15036839, -0.10442796, 0.025975846, -0.22467794, -0.17808884, -0.015909953, -0.041134287, -0.023751343, 0.1625684, 0.25692436, 0.0014141075, -0.08379793, -0.073593065, -0.01214905, -0.15444282, ...]"


In [ ]:
df_question1 = df_question[['embeddings']]
df_corpus1 = df_corpus[['embeddings']]

def find_most_similar_row(df_question1, df_corpus1, target_row_index):
  """
  Finds the most semantically similar row in df1 to the row at the given index in df2.

  Args:
    df1: The first dataframe.
    df2: The second dataframe.
    target_row_index: The index of the row in df2 to compare to.

  Returns:
    The index of the most similar row in df1 and its cosine similarity score.
  """
  target_embedding = df_question1.iloc[target_row_index].values
  similarities = np.dot(df_question1, target_embedding)
  most_similar_index = np.argmax(similarities)
  similarity_score = similarities[most_similar_index]
  return most_similar_index, similarity_score

# Example usage
target_row_index = 0  # Change this to the index of the row you want to compare in df2
most_similar_index, similarity_score = find_most_similar_row(df_question1, df_question1, target_row_index)
#print(f"Most similar row in df1: {most_similar_index} with similarity score: {similarity_score}")

In [ ]:

Have embedded corpus + question.
Need semantic search of corpus, then RAG on that.


In [12]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances_argmin_min
import ast

# Sample DataFrames with lists represented as strings
df1_data = {'column_name': [
    [0.090262967, 0.08659528, 0.12466674, -0.01663203, -0.12787767, -0.025562787, 0.031162905, -0.051343713],
    # Add more rows as needed
]}

df2_data = {'column_name': [
    [0.040262967, 0.08659528, 0.12466674, -0.01663203, -0.12787767, -0.025562787, 0.031162905, -0.051343713],
    [0.090262967, 0.08659528, 0.12466674, -0.01663203, -0.12787767, -0.025562787, 0.031162905, -0.051343713],
    [0.080262967, 0.08659528, 0.12466674, -0.01663203, -0.12787767, -0.025562787, 0.031162905, -0.051343713],
    [0.070262967, 0.08659528, 0.12466674, -0.01663203, -0.12787767, -0.025562787, 0.031162905, -0.051343713],
    [0.060262967, 0.08659528, 0.12466674, -0.01663203, -0.12787767, -0.025562787, 0.031162905, -0.051343713],
]}

df1 = pd.DataFrame(df1_data)
df2 = pd.DataFrame(df2_data)

# Calculate cosine similarity
similarity_matrix = cosine_similarity([df2['column_name'].iloc[0]], df1['column_name'].tolist())

#print(similarity_matrix)

# Find the index of the most similar row
top_index = pairwise_distances_argmin_min(similarity_matrix, df1['column_name'].tolist())[0][0]

print("Most similar row index in df1:", top_index)


ValueError: ignored

In [ ]:
df1 = df_question[['embeddings']]
df2 = df_corpus[['embeddings']]

def find_most_similar_row(df1, df2, target_row_index):
  """
  Finds the most semantically similar row in df1 to the row at the given index in df2.

  Args:
    df1: The first dataframe.
    df2: The second dataframe.
    target_row_index: The index of the row in df2 to compare to.

  Returns:
    The index of the most similar row in df1 and its cosine similarity score.
  """
  target_embedding = df2.iloc[target_row_index].values
  similarities = np.dot(df1, target_embedding)
  most_similar_index = np.argmax(similarities)
  similarity_score = similarities[most_similar_index]
  return similarities

# Example usage
target_row_index = 0  # Change this to the index of the row you want to compare in df2
similarities = find_most_similar_row(df1, df2, target_row_index)
#print(f"Most similar row in df1: {most_similar_index} with similarity score: {similarity_score}")
print(similarities)



[array([-3.67287640e-03, -2.61419831e-04,  4.75630024e-03,  5.51268226e-04,
        -1.93580370e-02, -1.34837732e-03,  3.63033637e-03,  9.00889339e-04,
        -5.77250402e-03, -4.91564348e-03,  1.96440751e-03,  1.36070065e-02,
         5.54181309e-03, -2.19885800e-02,  4.78684291e-04,  6.65812194e-03,
         1.33514369e-03, -5.11229225e-03, -1.72731851e-03,  2.18045039e-04,
        -9.13669239e-04,  3.35765560e-03,  1.60530105e-03, -8.85327754e-04,
         2.80713644e-02,  1.93883535e-02, -2.15009833e-03,  3.12626688e-03,
         2.23809853e-03,  9.61364945e-04, -5.39885461e-03,  5.54259773e-03,
        -8.71386728e-05,  6.97257509e-03,  1.04990602e-03,  1.52224535e-02,
         1.51265680e-03, -2.02796291e-05,  1.50647173e-02,  2.81080548e-02,
         1.06494967e-02,  3.37842316e-03, -2.41865474e-03, -2.38737339e-04,
         4.42275815e-02, -1.35200610e-03,  9.68195871e-03,  9.48409364e-03,
        -7.56371347e-03,  5.36933057e-02,  4.99705784e-03, -2.61060894e-02,
        -1.6

In [ ]:
# -- Functions -----
# (np.dot required based on embedding model)
# ref: https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro-textemb-vectorsearch.ipynb
def get_dot_product(row):
    return np.dot(row, query_vector)
def get_context_from_question(question, vector_store, sort_index_value=2):
    global query_vector
    query_vector = np.array(embedding_model_with_backoff([question]))
    top_matched = (vector_store["embedding"]
                   .apply(get_dot_product)
                   .sort_values(ascending=False)[:sort_index_value]
                   .index
                   )
    #top_matched_df = vector_store[vector_store.index.isin(top_matched)][["text", "embedding"]]
    top_matched_df = vector_store[vector_store.index.isin(top_matched)][["text", "embedding", "page", "TOC"]]

    # -- Can below two be removed?
    # -- (New 15/11) Current citatons duplicates context (i.e. once for each time it encounters a new section) so trialing this
    #top_matched_df = top_matched_df.drop_duplicates(subset=['text'], keep='first')
    context = " ".join(vector_store[vector_store.index.isin(top_matched)]["text"].values)
    return context, top_matched_df

context, top_matched_df = get_context_from_question(
    #question,
    resp0, # this is a change due to above (trialing the manual mqr in place of question)
    vector_store=df1,
    sort_index_value=4,  # Top N results to pick from embedding vector search
)

KeyError: ignored

In [17]:
!pip install faiss-cpu
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.5 MB/s eta 0:00:00


In [19]:
import os
import sys
# import pinecone
from langchain.llms import Replicate
from langchain.vectorstores import Pinecone
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS

# Replicate API token to access llama2
# create a replicate account and get the secret api token associated with your account
os.environ['REPLICATE_API_TOKEN'] = "use_replicate_api_token"
#PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')

def get_textchunks(pdf):
    loader = PyPDFLoader(pdf)
    documents = loader.load()
    # Split the documents into smaller chunks for processing
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    return texts

def get_vectorstore(text_chunks):
    #creating a vector index for the document
    embeddings = HuggingFaceEmbeddings()
    vectordb  = FAISS.from_documents(text_chunks, embeddings)
    return vectordb

def get_conv_chain(vectordb):
    # Initialize Replicate Llama2 Model
    llm = Replicate(
        model="a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
        input={"temperature": 0.75, "max_length": 3000}
    )

    chat_chain = ConversationalRetrievalChain.from_llm(
        llm,
        vectordb.as_retriever(search_kwargs={'k': 2}),
        return_source_documents=True)
    return chat_chain

if __name__ == "__main__":
    text_chunks = get_textchunks('/content/cba_fsg.pdf')
    vectordb = get_vectorstore(text_chunks)
    qa_chain = get_conv_chain(vectordb)
    chat_history = []
    while True:
        query = input('Prompt: ')
        if query.lower() in ["exit", "quit", "q"]:
            print('Exiting')
            sys.exit()
        result = qa_chain({'question': query, 'chat_history': chat_history})
        print('Answer: ' + result['answer'] + '\n')
        chat_history.append((query, result['answer']))

Prompt: what is a fsg


ImportError: ignored

In [ ]:
# Import necessary libraries
import os
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4Free

# Define document class
class DocGPT:
    def __init__(self, docs):
        self.docs = docs
        self.qa_chain = None
        self._llm = None

        # Define prompts for question answering and answer refinement
        self.prompt_template = (
            "Only answer what is asked. Answer step-by-step.\n"
            "... (instructions for summarizing and formatting)...\n"
            "{context}\n\n"
            "Question: {question}\n"
        )
        self.prompt = PromptTemplate(self.prompt_template, ["context", "question"])
        self.refine_prompt_template = (
            "Original question: {question}\n"
            "Existing answer: {existing_answer}\n"
            "Refine the answer with new context:\n"
            "------------\n"
            "{context_str}\n"
            "------------\n"
            "Refine the answer based on the new context. Use newlines appropriately and avoid double newlines.\n"
        )
        self.refine_prompt = PromptTemplate(self.refine_prompt_template, ["question", "existing_answer", "context_str"])

    @property
    def llm(self):
        return self._llm

    @llm.setter
    def llm(self, llm):
        self._llm = llm

    def _embeddings(self):
        try:
            # Use OpenAI embeddings if API key available
            embeddings = OpenAIEmbeddings()
        except:
            # Otherwise, use Hugging Face embeddings
            embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
            )

        # Create FAISS database from documents and embeddings
        db = FAISS.from_documents(documents=self.docs, embedding=embeddings)
        return db

    def create_qa_chain(self, chain_type="stuff", verbose=True):
        # Set up prompts and chain type kwargs
        chain_type_kwargs = {
            "question_prompt": self.prompt,
            "verbose": verbose,
            "refine_prompt": self.refine_prompt,
        }

        # Create document database and retriever
        db = self._embeddings()
        retriever = db.as_retriever()

        # Create RetrievalQA chain with GPT4Free LLM
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=GPT4Free(),
            chain_type=chain_type,
            retriever=retriever,
            chain_type_kwargs=chain_type_kwargs,
        )

    def run(self, query: str) -> str:
        response = "Nothing..."
        with get_openai_callback() as callback:
            if isinstance(self.qa_chain, RetrievalQA):
                response = self.qa_chain.run(query)
            logger.info(callback)
        return response


# GPT4Free LLM class with custom providers and testing
class GPT4Free(LLM):
    providers_table = {
        f"g4f.Provider.{provider}": getattr(g4f.Provider, provider)
        for provider in g4f.Provider.__all__
    }
    provider = "g4f.Provider.DeepAi"

    @property
    def _llm_type(self):
        return "gpt4free model"

    def _call(self, prompt: str, stop: Optional[List[str]] = None, run_manager: Optional[CallbackManagerForLLMRun] = None):
        try:
            # Use specified provider for GPT-3.5-turbo model
            provider = self.providers_table.get(self.provider, None)
            logger.info(f"Using provider: {provider}")
            return g4f.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                provider=provider,


AttributeError: ignored

In [ ]:
#https://infohub.delltechnologies.com/p/using-retrieval-augmented-generation-rag-on-a-custom-pdf-dataset-with-dell-technologies/